# GitHub Repository Metric Analysis

In this notebook, we will fetch the GitHub Issue/PR data for the repositories mentioned in [sigs.yaml](https://github.com/open-services-group/community/blob/main/sigs.yaml) using the [MI tool](https://github.com/thoth-station/mi), pre-process the raw data into suitable data frames and store them as parquet files to an s3 bucket. We will also create [Trino](https://trino.io/) tables for the generated parquet files so that we can later create dashboards in [Superset](https://superset.operate-first.cloud/).

This notebook will serve as a template notebook to analyze different GitHub repositories so that it can be easily executed in automation as part of our metrics processing pipeline. The notebook can be executed in parallel for different repos by passing as an argument the GitHub repository for which we would like to analyze and calculate metrics.

(Related issues: [Issue 1](https://github.com/open-services-group/metrics/issues/19))

In [1]:
import os
from dotenv import find_dotenv, load_dotenv
import warnings
import trino
from s3_communication import S3Communication
from github import Github
import pandas as pd

warnings.filterwarnings("ignore")
load_dotenv(find_dotenv())

True

In [2]:
## Create a .env file on your local with the correct configs
REPO = os.getenv("REPO")
ORG = os.getenv("ORG")
SIG = os.getenv("SIG")
GITHUB_ACCESS_TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")
s3_endpoint_url = os.getenv("S3_ENDPOINT")
aws_access_key_id = os.getenv("S3_ACCESS_KEY")
aws_secret_access_key = os.getenv("S3_SECRET_KEY")
s3_bucket = os.getenv("S3_BUCKET")
repo_slug = f"{ORG}/{REPO}"

In [3]:
# Note: The GitHub access token needs to be exported before importing the srcopmetrics package (current bug)
from srcopsmetrics.entities.issue import Issue  # noqa: E402
from srcopsmetrics.entities.pull_request import PullRequest  # noqa: E402

In [4]:
# init s3 connector
s3c = S3Communication(
    s3_endpoint_url, aws_access_key_id, aws_secret_access_key, s3_bucket
)

In [5]:
# Gather the data
!python -m srcopsmetrics.cli -clr $repo_slug -e Issue,PullRequest

INFO:srcopsmetrics.github_knowledge:Overall repositories found: 1
INFO:srcopsmetrics.bot_knowledge:######################## Analysing open-services-group/metrics ########################

INFO:srcopsmetrics.bot_knowledge:########################
INFO:srcopsmetrics.bot_knowledge:Detected entities:
CodeFrequency # Commit # DependencyUpdate # Fork # Issue # IssueEvent # KebechetUpdateManager # License # PullRequest # PullRequestDiscussion # RawIssue # RawPullRequest # ReadMe # Release # Stargazer # TrafficClones # TrafficPaths # TrafficPaths # TrafficReferrers # TrafficClones # TrafficViews
INFO:srcopsmetrics.bot_knowledge:########################
INFO:srcopsmetrics.bot_knowledge:Issue inspection
INFO:srcopsmetrics.entities.tools.storage:Loading knowledge locally
INFO:srcopsmetrics.entities.tools.storage:Data from file %s loaded
INFO:srcopsmetrics.entities.interface:Found previous Issue knowledge for open-services-group/metrics with 7 records
INFO:srcopsmetrics.iterator:-------------Issue

## Issue Metrics

Now, lets fetch the issues for the repository and derive some metrics.

In [6]:
issue = Issue(repo_slug)
issue_df = issue.load_previous_knowledge(is_local=True)
issue_df = issue_df.reset_index()
issue_df.head()

,id,title,body,created_by,created_at,closed_by,closed_at,labels,interactions
0,13,Collect metrics for all sub-projects in the OS...,Collect GitHub data using the MI tool i.e issu...,hemajv,1970-01-20 00:55:37.949,hemajv,1970-01-20 00:58:32.352,{},"{'oindrillac': 128, 'hemajv': 170, 'MichaelCli..."
1,10,Spike: Define OKR completion,This issue is created to discuss different app...,Shreyanand,1970-01-20 00:44:17.165,None,NaT,{},"{'MichaelClifford': 33, 'Shreyanand': 104, 'he..."
2,9,Project workflow diagram,Create an architecture diagram describing the ...,hemajv,1970-01-20 00:44:16.319,hemajv,1970-01-20 00:55:56.040,{},"{'oindrillac': 30, 'hemajv': 39, 'MichaelCliff..."
3,8,Update Readme,"Update the Readme to reflect project goals, re...",Shreyanand,1970-01-20 00:44:14.140,sesheta,1970-01-20 00:55:55.068,{},{}
4,7,[EPIC] Visualization,Create dashboards/reports to visualize metrics...,hemajv,1970-01-20 00:38:36.285,None,NaT,{},{}


In [7]:
# Retain only relevant columns
issue_cols_to_drop = ["labels", "interactions"]
issue_df = issue_df.drop(columns=issue_cols_to_drop)
issue_df["org"] = ORG
issue_df["repo"] = REPO

issue_df.head()

,id,title,body,created_by,created_at,closed_by,closed_at,org,repo
0,13,Collect metrics for all sub-projects in the OS...,Collect GitHub data using the MI tool i.e issu...,hemajv,1970-01-20 00:55:37.949,hemajv,1970-01-20 00:58:32.352,open-services-group,metrics
1,10,Spike: Define OKR completion,This issue is created to discuss different app...,Shreyanand,1970-01-20 00:44:17.165,None,NaT,open-services-group,metrics
2,9,Project workflow diagram,Create an architecture diagram describing the ...,hemajv,1970-01-20 00:44:16.319,hemajv,1970-01-20 00:55:56.040,open-services-group,metrics
3,8,Update Readme,"Update the Readme to reflect project goals, re...",Shreyanand,1970-01-20 00:44:14.140,sesheta,1970-01-20 00:55:55.068,open-services-group,metrics
4,7,[EPIC] Visualization,Create dashboards/reports to visualize metrics...,hemajv,1970-01-20 00:38:36.285,None,NaT,open-services-group,metrics


In [8]:
# Upload the processed df to s3 as a parquet file
s3c.upload_df_to_s3(
    df=issue_df,
    s3_prefix="open-services-group/metrics/github/issues",
    s3_key=f"{ORG}-{REPO}.parquet",
)

{'ResponseMetadata': {'RequestId': 'tx00000000000000007b9c8-00622a6baa-bd9943-ocs-storagecluster-cephobjectstore',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '0',
   'etag': '"244ad606c3438da06a0739d6dfa728f7"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx00000000000000007b9c8-00622a6baa-bd9943-ocs-storagecluster-cephobjectstore',
   'date': 'Thu, 10 Mar 2022 21:20:42 GMT',
   'set-cookie': 'bbdcd938787a45e68f8d240a4e2dadcf=9245b3fe660230b2beaa13e1023f5083; path=/; HttpOnly'},
  'RetryAttempts': 0},
 'ETag': '"244ad606c3438da06a0739d6dfa728f7"'}

## PR Metrics

Now, lets fetch the PRs for the repository and derive some metrics.

In [9]:
pr = PullRequest(repo_slug)
pr_df = pr.load_previous_knowledge(is_local=True)
pr_df = pr_df.reset_index()
pr_df.head()

,id,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at
0,20,Add repo analysis notebook,Adding a repo analysis template notebook\r\n\r...,XXL,hemajv,2022-02-25 19:37:55,2022-03-07 20:00:21,sesheta,2022-03-07 20:00:21,sesheta,1,2,"{'review-notebook-app[bot]': 29, 'MichaelCliff...","{'899535619': {'author': 'MichaelClifford', 'w...","[lgtm, size/XXL, approved]",[df21207d6580e3977af41cfde8cefc347e1d3085],"[notebooks/Repo_Analysis.ipynb, notebooks/s3_c...",2022-03-03 21:28:32,NaT
1,17,Add GitHub data analysis notebook,Resolves #16,XXL,hemajv,2022-02-17 19:17:47,2022-02-25 16:46:44,sesheta,2022-02-25 16:46:43,sesheta,1,3,"{'suppathak': 139, 'hemajv': 168, 'review-note...","{'892912080': {'author': 'Shreyanand', 'words_...","[lgtm, size/XXL, approved]",[1655de6634ab9f2676844929f3ef8bf7f60384ef],"[.pre-commit-config.yaml, docs/conf.py, notebo...",2022-02-24 19:40:11,NaT
2,15,Automatic update of dependencies by Kebechet f...,Kebechet has updated the dependencies to the l...,S,khebhut[bot],2022-02-15 20:28:05,NaT,None,NaT,None,1,1,"{'sesheta': 202, 'khebhut[bot]': 12}",{},"[size/S, needs-ok-to-test, bot]",[d4b232ee026e1f46d6c430d3a447ff3531afea84],[Pipfile.lock],NaT,NaT
3,14,Update README,Resolves #8,M,hemajv,2022-02-15 17:00:53,2022-02-15 19:57:48,sesheta,2022-02-15 19:57:48,sesheta,1,2,{'sesheta': 65},"{'883344363': {'author': 'oindrillac', 'words_...","[lgtm, size/M, approved]",[09a63cf6c1b8d058d14c3ca831be672232c4b91e],"[README.md, docs/imgs/project_architecture.jpeg]",2022-02-15 17:07:29,2022-02-15 19:56:42
4,12,Update OWNERS,cc @Shreyanand,S,hemajv,2022-02-09 18:07:46,2022-02-14 15:59:37,sesheta,2022-02-14 15:59:37,sesheta,1,1,"{'MichaelClifford': 11, 'sesheta': 65, 'harsha...",{},"[size/S, approved]",[6dff26159faa30e67808db422db9131fb67888e6],[OWNERS],NaT,NaT


In [10]:
# Retain only relevant columns
pr_cols_to_drop = ["interactions", "reviews", "labels", "commits", "changed_files"]
prs_df = pr_df.drop(columns=pr_cols_to_drop)
prs_df["org"] = ORG
prs_df["repo"] = REPO

prs_df.head()

,id,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,first_review_at,first_approve_at,org,repo
0,20,Add repo analysis notebook,Adding a repo analysis template notebook\r\n\r...,XXL,hemajv,2022-02-25 19:37:55,2022-03-07 20:00:21,sesheta,2022-03-07 20:00:21,sesheta,1,2,2022-03-03 21:28:32,NaT,open-services-group,metrics
1,17,Add GitHub data analysis notebook,Resolves #16,XXL,hemajv,2022-02-17 19:17:47,2022-02-25 16:46:44,sesheta,2022-02-25 16:46:43,sesheta,1,3,2022-02-24 19:40:11,NaT,open-services-group,metrics
2,15,Automatic update of dependencies by Kebechet f...,Kebechet has updated the dependencies to the l...,S,khebhut[bot],2022-02-15 20:28:05,NaT,None,NaT,None,1,1,NaT,NaT,open-services-group,metrics
3,14,Update README,Resolves #8,M,hemajv,2022-02-15 17:00:53,2022-02-15 19:57:48,sesheta,2022-02-15 19:57:48,sesheta,1,2,2022-02-15 17:07:29,2022-02-15 19:56:42,open-services-group,metrics
4,12,Update OWNERS,cc @Shreyanand,S,hemajv,2022-02-09 18:07:46,2022-02-14 15:59:37,sesheta,2022-02-14 15:59:37,sesheta,1,1,NaT,NaT,open-services-group,metrics


In [11]:
# Upload the processed df to s3 as a parquet file
s3c.upload_df_to_s3(
    df=prs_df,
    s3_prefix="open-services-group/metrics/github/prs",
    s3_key=f"{ORG}-{REPO}.parquet",
)

{'ResponseMetadata': {'RequestId': 'tx00000000000000007b9ca-00622a6bb0-bd9943-ocs-storagecluster-cephobjectstore',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '0',
   'etag': '"1e6c5d3678f53bdeae71754ba0b8e7c4"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx00000000000000007b9ca-00622a6bb0-bd9943-ocs-storagecluster-cephobjectstore',
   'date': 'Thu, 10 Mar 2022 21:20:48 GMT',
   'set-cookie': 'bbdcd938787a45e68f8d240a4e2dadcf=9245b3fe660230b2beaa13e1023f5083; path=/; HttpOnly'},
  'RetryAttempts': 0},
 'ETag': '"1e6c5d3678f53bdeae71754ba0b8e7c4"'}

## Contributors metrics
Next, lets fetch the events for the repository. This table will be used to find information about contributors, their affiliations and the events they generate in the repository. 

In [12]:
# To do
# In the current version we can only get data for the last 90 days or 300 events
# limited by the guithub API
# Add a loop for getting older data
# Probably, get all events for a month and then loop to the next month

In [13]:
## Get internal members of OSG group
gth_obj = Github(GITHUB_ACCESS_TOKEN)
osg = gth_obj.get_organization("open-services-group")
internal = [m.login for m in osg.get_members()]

In [14]:
repo = gth_obj.get_repo(repo_slug)

In [15]:
# Define events we are interested in
issue_event_types = ["IssueCommentEvent", "IssuesEvent"]

pr_event_types = [
    "PullRequestEvent",
    "PullRequestReviewEvent",
    "PullRequestReviewCommentEvent",
]

In [16]:
# Create the events data frame
rows = []
for e in repo.get_events():
    if e.type in issue_event_types or e.type in pr_event_types:
        event_id = e.id
        created_at = e.created_at
        event_contributor_id = e.actor.id
        event_contributor = e.actor.login
        event_type = e.type
        event_action = e.payload["action"]
        if event_type in issue_event_types:
            issue_or_pr_id = e.payload["issue"]["number"]
        else:
            issue_or_pr_id = e.payload["pull_request"]["number"]
        rows.append(
            [
                event_id,
                created_at,
                event_contributor_id,
                event_contributor,
                event_type,
                event_action,
                issue_or_pr_id,
            ]
        )

In [17]:
# Add column names for events data frame
column_name = [
    "id",
    "created_at",
    "contributor_id",
    "contributor_name",
    "type",
    "action",
    "issue_or_pr_id",
]

In [18]:
# Add other required colums
events_df = pd.DataFrame(data=rows, columns=column_name)
events_df["org"] = ORG
events_df["repo"] = REPO
events_df["sig"] = SIG
events_df["internal_contributor"] = events_df["contributor_name"].apply(
    lambda x: x in internal
)

In [19]:
events_df.head()

,id,created_at,contributor_id,contributor_name,type,action,issue_or_pr_id,org,repo,sig,internal_contributor
0,20675625842,2022-03-10 08:59:40,33906690,sesheta,IssueCommentEvent,created,24,open-services-group,metrics,sig-data-science,True
1,20675625438,2022-03-10 08:59:39,89909507,schwesig,IssueCommentEvent,created,24,open-services-group,metrics,sig-data-science,True
2,20655246647,2022-03-09 09:48:47,89909507,schwesig,IssueCommentEvent,created,24,open-services-group,metrics,sig-data-science,True
3,20644931142,2022-03-08 20:05:32,8916126,Shreyanand,IssuesEvent,opened,24,open-services-group,metrics,sig-data-science,True
4,20644735935,2022-03-08 19:52:51,8916126,Shreyanand,IssueCommentEvent,created,23,open-services-group,metrics,sig-data-science,True


In [20]:
# Upload the processed df to s3 as a parquet file
s3c.upload_df_to_s3(
    df=events_df,
    s3_prefix="open-services-group/metrics/github/events",
    s3_key=f"{ORG}-{REPO}.parquet",
)

{'ResponseMetadata': {'RequestId': 'tx00000000000000007b9cd-00622a6bbc-bd9943-ocs-storagecluster-cephobjectstore',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '0',
   'etag': '"10646f6fde420f0c153e8c7bfad8ab66"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx00000000000000007b9cd-00622a6bbc-bd9943-ocs-storagecluster-cephobjectstore',
   'date': 'Thu, 10 Mar 2022 21:21:00 GMT',
   'set-cookie': 'bbdcd938787a45e68f8d240a4e2dadcf=9245b3fe660230b2beaa13e1023f5083; path=/; HttpOnly'},
  'RetryAttempts': 0},
 'ETag': '"10646f6fde420f0c153e8c7bfad8ab66"'}

## Create Trino Tables

Now that we have the processed data frames stored as parquet files in s3, we can generate [Trino](https://trino.io/) tables from it so that interactive dashboards can be implemented in [Superset](https://superset.apache.org/). We will be connecting to the [Operate First Trino](https://trino.operate-first.cloud/).

In [21]:
# Map the datatype columns of df to suitable datatype supported in Trino/Superset
_p2smap = {
    "object": "varchar",
    "int64": "bigint",
    "float64": "double",
    "datetime64[ns]": "timestamp",
    "bool": "boolean",
}


def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))


# Generate the Trino table schema
def generate_table_schema_pairs(df):
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0], t=e[1]) for e in pz])

In [22]:
# Create a Trino client
conn = trino.dbapi.connect(
    auth=trino.auth.BasicAuthentication(
        os.environ["TRINO_USER"], os.environ["TRINO_PASSWD"]
    ),
    host=os.environ["TRINO_HOST"],
    port=int(os.environ["TRINO_PORT"]),
    http_scheme="https",
    verify=True,
)
cur = conn.cursor()

In [23]:
# Check if Trino connection was successful
cur.execute("show catalogs")
cur.fetchall()[1]

['data_science_general']

In [24]:
# Create the issues table with data populated from parquet file
issue_schema = generate_table_schema_pairs(issue_df)

tabledef = """create table if not exists data_science_general.default.issues(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{s3_bucket}/open-services-group/metrics/github/issues'
)""".format(
    schema=issue_schema,
    s3_bucket=os.environ["S3_BUCKET"],
)

cur.execute(tabledef)
cur.fetchall()

[[True]]

In [25]:
# Create the PR table with data populated from parquet file
pr_schema = generate_table_schema_pairs(prs_df)

tabledef = """create table if not exists data_science_general.default.prs(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{s3_bucket}/open-services-group/metrics/github/prs'
)""".format(
    schema=pr_schema,
    s3_bucket=os.environ["S3_BUCKET"],
)

cur.execute(tabledef)
cur.fetchall()

[[True]]

In [26]:
# Create the PR table with data populated from parquet file
events_schema = generate_table_schema_pairs(events_df)

tabledef = """create table if not exists data_science_general.default.events(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{s3_bucket}/open-services-group/metrics/github/events'
)""".format(
    schema=events_schema,
    s3_bucket=os.environ["S3_BUCKET"],
)

cur.execute(tabledef)
cur.fetchall()

[[True]]

## Conclusion

In this notebook we:

- Fetched GitHub Issue/PR data for a specified org/repo using the MI `srcopsmetrics` module
- Pre-processed the raw data into data frames with relevant columns
- Uploaded the processed data frames as parquet files to an S3 bucket
- Created suitable tables for the parquet files generated in Trino

We can now further explore the GitHub data obtained for different repos/orgs and create interactive visualization dashboards in [Superset](https://superset.operate-first.cloud/).